In [87]:
import itertools
import pandas as pd
import numpy as np

In [23]:
def read_fasta_file(path):
    '''
    used for load fasta data and transformd into numpy.array format
    '''
    fh=open(m6a_benchmark_dataset)
    seq=[]
    for line in fh:
        if line.startswith('>'):
            continue
        else:
            seq.append(line.replace('\r\n',''))
    fh.close()
    matrix_data=np.array([list(e) for e in seq])
    return matrix_data

In [149]:
def AthMethPre_extract_one_line(data_line):
    '''
    extract features from one line, such as one m6A sample
    '''
    A=[0,0,0,1]
    U=[0,0,1,0]
    C=[0,1,0,0]
    G=[1,0,0,0]
    N=[0,0,0,0]
    feature_representation={"A":A,"U":U,"C":C,"G":G,"N":N}
    beginning=0
    end=len(data_line)-1
    one_line_feature=[]
    alphabet='AUCG'
    matrix_two=["".join(e) for e in itertools.product(alphabet, repeat=2)] # AA AU AC AG UU UC ...
    matrix_three=["".join(e) for e in itertools.product(alphabet, repeat=3)]# AAA AAU AAC ...
    matrix_four=["".join(e) for e in itertools.product(alphabet, repeat=4)]# AAAA AAAU AAAC ...
    feature_two=np.zeros(16)
    feature_three=np.zeros(64)
    feature_four=np.zeros(256)
    for index,data in enumerate(data_line):
        if index==beginning or index==end:
            one_line_feature.extend(feature_representation["N"])
        elif data in feature_representation.keys():
            one_line_feature.extend(feature_representation[data])
        if "".join(data_line[index:(index+2)]) in matrix_two and index <= end-1:
            feature_two[matrix_two.index("".join(data_line[index:(index+2)]))]+=1
        if "".join(data_line[index:(index+3)]) in matrix_three and index <= end-2:
            feature_three[matrix_three.index("".join(data_line[index:(index+3)]))]+=1
        if "".join(data_line[index:(index+4)]) in matrix_four and index <=end-3:
            feature_four[matrix_four.index("".join(data_line[index:(index+4)]))]+=1
    sum_two=np.sum(feature_two)
    sum_three=np.sum(feature_three)
    sum_four=np.sum(feature_four)
    one_line_feature.extend(feature_two/sum_two)
    one_line_feature.extend(feature_three/sum_three)
    one_line_feature.extend(feature_four/sum_four)
    return one_line_feature


In [150]:
def AthMethPre_feature_extraction(matrix_data):
    final_feature_matrix=[AthMethPre_extract_one_line(e) for e in matrix_data]
    return final_feature_matrix

In [151]:

m6a_benchmark_dataset='/home02/chenhuangrong/m6a_data.txt'
matrix_data=read_fasta_file(m6a_benchmark_dataset)
final_feature_matrix=AthMethPre_feature_extraction(matrix_data)
pd.DataFrame(final_feature_matrix).to_csv('/home02/chenhuangrong/AthMethPre.csv',header=None,index=False)


